### Implement Neural Style transfer algorithm



- Use a VGG19 network to compute layer activations for the style-reference image, the target image and the generated image at the same time.
- Use the layer activations for these 3 images to define loss functions
- Use Gradient Descent to minimize these loss functions

In [0]:
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras.applications import vgg19
from keras import backend as K
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Define intial variables

In [0]:
target_image_path = '/content/gdrive/My Drive/Colab Notebooks/data/neural-style-transfer/target_reference_image.jpg'
style_reference_image_path = '/content/gdrive/My Drive/Colab Notebooks/data/neural-style-transfer/style_reference_image.jpg'

width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width*img_height/height)

Define auxiliary functions for loading, preprocessing and postprocessing the images that go in and out of the VGG19 convnet.

In [0]:
def preprocess_image(image_path):
  img = load_img(image_path, target_size=(img_height,img_width))
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = vgg19.preprocess_input(img)
  return img

def deprocess_image(x):
  x[:,:,0] += 103.939
  x[:,:,1] += 116.779
  x[:,:,2] += 123.68
  x = x[:,:,::-1]
  x = np.clip(x,0,255).astype('uint8')
  return x

Let's setup the VGG19 network. It takes as input three images: the style reference
image, the target image, and a placeholder that will contain the generated
image. The style-reference and target image are static whereas the values contained in the placeholder of the generated image will change over time.

In [0]:
target_image = K.constant(preprocess_image(target_image_path))
style_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1,img_height,img_width,3))

#Combines the three images in a single batch
input_tensor = K.concatenate([target_image, style_image,combination_image],axis=0)

#Builds the VGG19 network with the batch of three images as input. The model will be loaded with pretrained ImageNet weights.
model = vgg19.VGG19(input_tensor = input_tensor, weights='imagenet',include_top=False)
print('Model loaded')

Model loaded


Lets define Content and Style loss.
- Content loss : ensures the top layer of the VGG19 convnet has a similar view of the target image and the generated image.
- Style loss : computes Gram matrix of a map of the correlations found in the original feature matrix

In [0]:
def content_loss(base,combination):
  return K.sum(K.square(combination-base))

def gram_matrix(x):
  "Auxiliary function to help compute gram matrix for the style loss"
  features = K.batch_flatten(K.permute_dimensions(x,(2,0,1)))
  gram = K.dot(features,K.transpose(features))
  return gram

def style_loss(style,combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height * img_width
  return K.sum(K.square(S-C))/(4. * (channels**2)*(size**2))
  

Add Total variation loss which is like a regularization loss allowing for spatial continuity in the generated image, thus avoiding overly pixelated results.

In [0]:
def total_variation_loss(x):
  a = K.square(
  x[:,:img_height-1,:img_width-1,:] - 
  x[:,1:,:img_width-1,:])
  
  b = K.square(x[:,:img_height-1,:img_width-1,:] - 
  x[:,:img_height-1,1:,:])
  
  return K.sum(K.pow(a+b,1.25))

The loss that we minimize is a weighted average of these three losses. To compute the
content loss, we use—the block5_conv2 layer—whereas for the style loss, we use a list of layers than spans both low-level and high-level layers. We add the total variation loss at the end.

In [0]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

#Layers used for style and content loss
content_layer = 'block5_conv2'
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']

#Weights in weighted average of the loss components
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.variable(0.) #variable to which we add all the losses

#Add content loss
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0,:,:,:]
combination_features = layer_features[2,:,:,:]
loss += content_weight * content_loss(target_image_features, combination_features)

#Add style loss
for layer_name in style_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1,:,:,:]
  combination_features = layer_features[2,:,:,:]
  
  s1 = style_loss(style_reference_features,combination_features)
  loss += (style_weight/len(style_layers)) * s1
  
loss += total_variation_weight * total_variation_loss(combination_image)

Setup the gradient ascent process using L-BFGS algorithm

In [0]:
#Gets the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)[0]

#function to fetch the values of the current loss and the current gradients
fetch_loss_and_grads = K.function([combination_image],[loss,grads])

class Evaluator(object):
  """This class wraps fetch_loss_and_grads in a way that lets you retrieve the losses and gradients via
   two separate method calls, which is required by the SciPy optimizer"""
  
  def __init__(self):
    self.loss_value = None
    self.grads_value = None
    
  def loss(self,x):
    assert self.loss_value is None
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    self.loss_value = loss_value
    self.grad_values = grad_values
    return self.loss_value
  
  def grads(self,x):
    assert self.loss_value is not None
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_values = None
    return grad_values
  
evaluator = Evaluator()
    

Now we run the gradient ascent process using SciPy's L-BFGS algorithm

In [0]:
result_prefix = '/content/gdrive/My Drive/Colab Notebooks/data/neural-style-transfer/my_result'
iterations = 30

x = preprocess_image(target_image_path) #initial state

#flatten the image because scipy.optimize.fmin_l_bfgs_b can only process flat vectors.
x = x.flatten()  
for i in range(iterations):
  print ('Start of iteration', i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
  
  print('Current loss value:', min_val)
  img = x.copy().reshape((img_height, img_width, 3))
  img = deprocess_image(img)
  fname = result_prefix + '_at_iteration_%d.png' %i
  imsave(fname,img)
  
  print('Image saved as', fname)
  end_time = time.time()
  print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 5263411700.0
Image saved as /content/gdrive/My Drive/Colab Notebooks/data/neural-style-transfer/my_result_at_iteration_0.png
Iteration 0 completed in 12s
Start of iteration 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 1875334300.0
Image saved as /content/gdrive/My Drive/Colab Notebooks/data/neural-style-transfer/my_result_at_iteration_1.png
Iteration 1 completed in 8s
Start of iteration 2
Current loss value: 1063689600.0
Image saved as /content/gdrive/My Drive/Colab Notebooks/data/neural-style-transfer/my_result_at_iteration_2.png
Iteration 2 completed in 8s
Start of iteration 3
Current loss value: 756387260.0
Image saved as /content/gdrive/My Drive/Colab Notebooks/data/neural-style-transfer/my_result_at_iteration_3.png
Iteration 3 completed in 8s
Start of iteration 4
Current loss value: 601453760.0
Image saved as /content/gdrive/My Drive/Colab Notebooks/data/neural-style-transfer/my_result_at_iteration_4.png
Iteration 4 completed in 8s
Start of iteration 5
Current loss value: 478343580.0
Image saved as /content/gdrive/My Drive/Colab Notebooks/data/neural-style-transfer/my_result_at_iteration_5.png
Iteration 5 completed in 8s
Start of iteration 6
Current loss value: 396649000.0
I